In [16]:
%load_ext autoreload
%autoreload 2
%aimport anndata
%aimport os
%aimport torch
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from typing import Dict, Union, List, Tuple
from torch.utils.data import Dataset
# Control UMAP numba warnings
import warnings; warnings.simplefilter('ignore')
import copy
import scvi
from scvi.dataset.cortex import CortexDataset
from scvi.models import VAE
from scvi.inference import UnsupervisedTrainer, load_posterior
from scvi import set_seed
from scvi.dataset import setup_anndata

#STUFF THAT I MESSED AROUND WITH
# from BioDataset import BioDataset
# from constants import X_KEY, BATCH_KEY, LOCAL_L_MEAN_KEY, LOCAL_L_VAR_KEY, LABELS_KEY

# Sets torch and numpy random seeds, run after all scvi imports
set_seed(0)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from scvi.dataset.dataset import   GeneExpressionDataset
from scvi.dataset.anndataset import DownloadableAnnDataset
save_path = "data/"
dataset1 = DownloadableAnnDataset(
    "pbmc_10k_protein.h5ad",
    save_path=save_path,
    url="https://github.com/YosefLab/scVI-data/raw/master/pbmc_10k_protein_v3.h5ad?raw=true",
    cell_measurements_col_mappings={"protein_expression":"protein_names"},
)

dataset2 = DownloadableAnnDataset(
    "pbmc_5k_protein.h5ad",
    save_path=save_path,
    url="https://github.com/YosefLab/scVI-data/raw/master/pbmc_5k_protein_v3.h5ad?raw=true",
    cell_measurements_col_mappings={"protein_expression":"protein_names"},
)

[2020-06-26 12:08:00,815] INFO - scvi.dataset.dataset | File /Users/galen/scVI/galen/data/pbmc_10k_protein.h5ad already downloaded
[2020-06-26 12:08:02,863] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-26 12:08:02,864] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-26 12:08:04,307] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-26 12:08:04,675] INFO - scvi.dataset.dataset | Downsampled from 6855 to 6855 cells
[2020-06-26 12:08:04,724] INFO - scvi.dataset.dataset | File /Users/galen/scVI/galen/data/pbmc_5k_protein.h5ad already downloaded
[2020-06-26 12:08:05,990] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-26 12:08:05,991] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-26 12:08:06,822] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-26 12:08:07,035] INFO - scvi.dataset.dataset | Downsampled from 3994 to 3994 cells


In [20]:
from scvi.models import TOTALVI
from scvi.inference import TotalPosterior, TotalTrainer

In [21]:
# concatenate the datasets
dataset = GeneExpressionDataset()
dataset.populate_from_datasets([dataset1, dataset2])
dataset.subsample_genes(4000)

[2020-06-26 12:08:07,137] INFO - scvi.dataset.dataset | Merging datasets. Input objects are modified in place.
[2020-06-26 12:08:07,138] INFO - scvi.dataset.dataset | Gene names and cell measurement names are assumed to have a non-null intersection between datasets.
[2020-06-26 12:08:07,164] INFO - scvi.dataset.dataset | Keeping 15792 genes
[2020-06-26 12:08:09,576] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-26 12:08:10,969] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-26 12:08:10,971] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-26 12:08:12,297] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-26 12:08:12,933] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-26 12:08:12,934] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-26 12:08:15,975] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-26 12:08:15,976] 

In [22]:
adata = dataset.to_anndata()

In [23]:
setup_anndata(adata, protein_expression_obsm_key = 'protein_expression', batch_key = 'batch_indices')

[2020-06-26 12:08:26,359] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-06-26 12:08:26,360] INFO - scvi.dataset._anndata | Using batches from adata.obs["batch_indices"]
[2020-06-26 12:08:26,361] INFO - scvi.dataset._anndata | No label_key inputted, assuming all cells have same label
[2020-06-26 12:08:26,364] INFO - scvi.dataset._anndata | Computing library size prior per batch
[2020-06-26 12:08:26,446] INFO - scvi.dataset._anndata | Successfully registered anndata object containing 10849 cells, 4000 genes, and 2 batches 
Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels', 'protein_expression']


In [24]:
test_mode = False

totalvae = TOTALVI(
    adata.uns['scvi_summary_stats']['n_genes'], 
    adata.uns['scvi_summary_stats']['n_proteins'], 
    0
#     adata.uns['scvi_summary_stats']['n_batch'],
)
use_cuda = True
lr = 4e-3
n_epochs = 500

# totalVI is trained on 90% of the data
# Early stopping does not comply with our automatic notebook testing so we disable it when testing
trainer = TotalTrainer(
    totalvae,
    adata,
    train_size=0.90,
    test_size=0.10,
    use_cuda=use_cuda,
    frequency=1,
    batch_size=256,
    early_stopping_kwargs="auto" if not test_mode else None
)

In [25]:
trainer.train(n_epochs = 10)

[2020-06-26 12:08:30,579] INFO - scvi.inference.inference | KL warmup phase may exceed overall training phase.If your applications rely on posterior quality, consider training for more epochs or reducing the kl warmup.
[2020-06-26 12:08:30,579] INFO - scvi.inference.inference | KL warmup for 8136.75 iterations



[2020-06-26 12:10:52,206] INFO - scvi.inference.inference | Training is still in warming up phase. If your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.
